In [163]:
import requests
from bs4 import BeautifulSoup
import mysql.connector
from mysql.connector import errorcode
import time
from datetime import datetime
import re
import pandas as pd
import csv

# Database connection
conn = mysql.connector.connect(
   user= "wsa",
   host= "34.68.250.121",
   database = "ufcElo",
   password = "LeBron>MJ!"
)
cursor = conn.cursor()

In [164]:
# Function to insert events
def insert_event(name, date):
    cursor.execute("INSERT INTO events (name, date) VALUES (%s, %s)", (name, date))
    conn.commit()
    return cursor.lastrowid

In [165]:
def insert_fight(event_id, winner_id, loser_id, weight_class, title_fight, method, round_end, fight_time, outcome, url):
    try:
        cursor.execute("""
            INSERT INTO fights (event_id, fighter1_id, fighter2_id, weight_class, title_fight, method, rounds, time, outcome, url)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (event_id, winner_id, loser_id, weight_class, title_fight, method, round_end, fight_time, outcome, url))
        conn.commit()  # ✅ Explicitly commit after insert
    except mysql.connector.Error as err:
        print("Error inserting fight:", err)
        conn.rollback()  # 🚨 Rollback if there's an issue

In [166]:
def insert_fighter(fighter_name, weight_class):
    cursor.execute("SELECT fighter_id FROM fighters WHERE name = %s", (fighter_name,))
    result = cursor.fetchone()

    if result:
        return result[0]
    else:
        cursor.execute("""
            INSERT INTO fighters (name, weightclass, wins, losses, draws, currentElo, highestElo)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (fighter_name, weight_class, 0, 0, 0, 1500, 1500))  # Default Elo is 1500
        conn.commit()
        return cursor.lastrowid

In [167]:
# Function to update Elo ratings in the database
def update_elo(winner_id, loser_id, is_title_fight):
    # Fetch current Elo ratings from the database
    cursor.execute("SELECT currentElo FROM fighters WHERE fighter_id = %s", (winner_id,))
    winner_elo = cursor.fetchone()[0]
    
    cursor.execute("SELECT currentElo FROM fighters WHERE fighter_id = %s", (loser_id,))
    loser_elo = cursor.fetchone()[0]
    
    # Expected score function
    def expected_score(rating_a, rating_b):
        return 1 / (1 + 10 ** ((rating_b - rating_a) / 400))
    
    # Elo constants
    k_factor = 50
    
    # Calculate expected scores
    expected_winner = expected_score(winner_elo, loser_elo)
    expected_loser = expected_score(loser_elo, winner_elo)
    
    # Update Elo based on the fight result
    if is_title_fight:
        winner_new_elo = winner_elo + (k_factor * (1 - expected_winner) * 2)
        loser_new_elo = loser_elo + k_factor * (0 - expected_loser)
    else:
        winner_new_elo = winner_elo + k_factor * (1 - expected_winner)
        loser_new_elo = loser_elo + k_factor * (0 - expected_loser)
    
    # Update current Elo for both fighters in the database
    cursor.execute("UPDATE fighters SET currentElo = %s WHERE fighter_id = %s", (winner_new_elo, winner_id))
    cursor.execute("UPDATE fighters SET currentElo = %s WHERE fighter_id = %s", (loser_new_elo, loser_id))

    # Update the highest Elo (all-time Elo) if necessary for winner
    cursor.execute("SELECT highestElo FROM fighters WHERE fighter_id = %s", (winner_id,))
    current_alltime_elo = cursor.fetchone()[0]
    if winner_new_elo > current_alltime_elo:
        cursor.execute("UPDATE fighters SET highestElo = %s WHERE fighter_id = %s", (winner_new_elo, winner_id))
    
    # Update the highest Elo (all-time Elo) if necessary for loser
    cursor.execute("SELECT highestElo FROM fighters WHERE fighter_id = %s", (loser_id,))
    current_alltime_elo = cursor.fetchone()[0]
    if loser_new_elo > current_alltime_elo:
        cursor.execute("UPDATE fighters SET highestElo = %s WHERE fighter_id = %s", (loser_new_elo, loser_id))

    conn.commit()

In [168]:

def load_fight_data_from_csv(file_path):
    fight_data = []
    with open(file_path, mode='r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header row
        for row in reader:
            fight_data.append(row)
    return fight_data

In [169]:
def store_fight_stats(fight_id, fighter_id, stats):
    print(stats)
    def sanitize(value):
        # Replace non-numeric values like '--' with NULL
        print(f"Sanitizing value: {value}")
        if value == '--':
            return 'NULL'
        if ':' in value:
            try:
                minutes, seconds = map(int, value.split(':'))
                return minutes * 60 + seconds  # Convert to total seconds
            except ValueError:
                return 'NULL' # In case there's an invalid time format
        # Remove '%' sign if it exists and convert to a float
        if '%' in value:
            return value.replace('%', '')
        try:
            # Try converting the value to a float
            return float(value)
        except ValueError:
            return 'NULL'  # Default to NULL if conversion fails

    # Prepare sanitized values for the SQL query
    kd = sanitize(stats['kd'])
    sig_str = sanitize(stats['sig_str'])
    sig_str_pct = sanitize(stats['sig_str_pct'])
    total_str = sanitize(stats['total_str'])
    td = sanitize(stats['td'])
    td_pct = sanitize(stats['td_pct'])
    sub_att = sanitize(stats['sub_att'])
    rev = sanitize(stats['rev'])
    ctrl = sanitize(stats['ctrl'])
    head = sanitize(stats['head'])
    body = sanitize(stats['body'])
    leg = sanitize(stats['leg'])
    distance = sanitize(stats['distance'])
    clinch = sanitize(stats['clinch'])
    ground = sanitize(stats['ground'])

    # Example SQL insert statement with sanitized values
    query = f"""
    INSERT INTO advanced_stats (fight_id, fighter_id, kd, sig_str, sig_str_pct, total_str, td, td_pct, sub_att, rev, ctrl, head, body, leg, distance, clinch, ground)
    VALUES ({fight_id}, {fighter_id}, {kd}, {sig_str}, {sig_str_pct}, {total_str}, {td}, {td_pct}, {sub_att}, {rev}, {ctrl}, {head}, {body}, {leg}, {distance}, {clinch}, {ground})
    """
    # Execute the query to store the stats in the database
    cursor.execute(query)
    print("Storing stats:", query)
    

In [170]:
def get_fight_stats(fight_url, fight_id, fighter1_id, fighter2_id):
    url = fight_url
    headers = {"User-Agent": "Mozilla/5.0"}

    # Fetch the webpage
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # Find all tables
    tables = soup.find_all("table")

    fight_stats_table = tables[0]  # Replace with the correct table index
    fight_stats_table2 = tables[2]  # Replace with the correct table index
    rows = fight_stats_table.find_all("tr")
    rows2 = fight_stats_table2.find_all("tr")
    fighter_stats = {
        'fighter1': {},
        'fighter2': {}
    }

    fighter_stats = {
        'fighter1': {'fighter_id': fighter1_id, 'stats': {}},
        'fighter2': {'fighter_id': fighter2_id, 'stats': {}}
    }

    # Process the stats from fight_stats_table
    for row in rows:
        cols = row.find_all("td")
        if len(cols) > 1:
            # Process the fighter names and stats
            fighter_names = [name.strip() for name in cols[0].text.split("\n") if name.strip()]
            if len(fighter_names) == 2:
                fighter1_name = fighter_names[0].strip()
                fighter2_name = fighter_names[1].strip()

                # Stats for fighter1
                stats = [col.text.strip().replace("\n", " ") for col in cols[1:]]
                fighter_stats['fighter1']['stats'] = {
                    'kd': stats[0].split()[0] if stats[0] != '--' else None,
                    'sig_str': stats[1].split()[0] if stats[1] != '--' else None,
                    'sig_str_pct': stats[2].split()[0] if stats[2] != '--' else None,
                    'total_str': stats[3].split()[0] if stats[3] != '--' else None,
                    'td': stats[4].split()[0] if stats[4] != '--' else None,
                    'td_pct': stats[5].split()[0] if stats[5] != '--' else None,
                    'sub_att': stats[6].split()[0] if stats[6] != '--' else None,
                    'rev': stats[7].split()[0] if stats[7] != '--' else None,
                    'ctrl': stats[8].split()[0]
                }

                # Stats for fighter2 (same as fighter1, but second stat)
                fighter_stats['fighter2']['stats'] = {
                    'kd': stats[0].split()[1] if stats[0] != '--' else None,
                    'sig_str': stats[1].split()[3] if stats[1] != '--' else None,
                    'sig_str_pct': stats[2].split()[1] if stats[2] != '--' else None,
                    'total_str': stats[3].split()[3] if stats[3] != '--' else None,
                    'td': stats[4].split()[3] if stats[4] != '--' else None,
                    'td_pct': stats[5].split()[1] if stats[5] != '--' else None,
                    'sub_att': stats[6].split()[1] if stats[6] != '--' else None,
                    'rev': stats[7].split()[1] if stats[7] != '--' else None,
                    'ctrl': stats[8].split()[1]
                }

    # Process advanced stats (from fight_stats_table2)
    for row in rows2:
        cols = row.find_all("td")
        if len(cols) > 1:
            # Process the advanced stats for each fighter
            stats = [col.text.strip().replace("\n", " ") for col in cols[1:]]
            fighter_stats['fighter1']['stats'].update({
                'head': stats[2].split()[0] if stats[2] != '--' else None,
                'body': stats[3].split()[0] if stats[3] != '--' else None,
                'leg': stats[4].split()[0] if stats[4] != '--' else None,
                'distance': stats[5].split()[0] if stats[5] != '--' else None,
                'clinch': stats[6].split()[0] if stats[6] != '--' else None,
                'ground': stats[7].split()[0] if stats[7] != '--' else None
            })

            fighter_stats['fighter2']['stats'].update({
                'head': stats[2].split()[3] if stats[2] != '--' else None,
                'body': stats[3].split()[3] if stats[3] != '--' else None,
                'leg': stats[4].split()[3] if stats[4] != '--' else None,
                'distance': stats[5].split()[3] if stats[5] != '--' else None,
                'clinch': stats[6].split()[3] if stats[6] != '--' else None,
                'ground': stats[7].split()[3] if stats[7] != '--' else None
            })
    print("Fighter Stats for Fighter 1:", fighter_stats['fighter1']['stats'])
    print("Fighter Stats for Fighter 2:", fighter_stats['fighter2']['stats'])
    store_fight_stats(fight_id, fighter_stats['fighter1']['fighter_id'], fighter_stats['fighter1']['stats'])
    store_fight_stats(fight_id, fighter_stats['fighter2']['fighter_id'], fighter_stats['fighter2']['stats'])
    conn.commit()
    #print(fighter_stats)    store_fight_stats(fight_id, fighter_stats['fighter2']['fighter_id'], fighter_stats['fighter2']['stats'])
    #return fighter_stats
    #return fighter_stats
#get_fight_stats("http://www.ufcstats.com/fight-details/9779eb1aae1d4654")

In [171]:
def update_fighter_record(fighter_id, outcome):
    if outcome == "win":
        cursor.execute("UPDATE fighters SET wins = wins + 1 WHERE fighter_id = %s", (fighter_id,))
    elif outcome == "loss":
        cursor.execute("UPDATE fighters SET losses = losses + 1 WHERE fighter_id = %s", (fighter_id,))
    elif outcome == "draw":
        cursor.execute("UPDATE fighters SET draws = draws + 1 WHERE fighter_id = %s", (fighter_id,))
    
    conn.commit()

In [172]:
def get_fight_id_from_url(fight_url):

    cursor.execute("SELECT fight_id FROM fights WHERE url = %s", (fight_url,))
    
    # Fetch the result (it should return a single row with fight_id)
    result = cursor.fetchone()
    
    if result:
        return result[0]  # Return the fight_id
    else:
        return None  # Return None if the fight_url is not found

In [173]:
# Function to process each fight
def process_fights(fight_data):
    for fight in fight_data:
        try:
            # Extract basic fight information
            winner = fight[1]
            loser = fight[2]
            weight_class = fight[3]
            title_fight = fight[4] == "Yes"
            method = fight[5]
            round_end = fight[6]
            fight_time = fight[7]
            outcome = fight[8]
            url = fight[9]

            # Insert fighters into the database and get their IDs
            winner_id = insert_fighter(winner, weight_class)  # Insert or get fighter ID for winner
            loser_id = insert_fighter(loser, weight_class)  # Insert or get fighter ID for loser
            
            # Insert the fight into the database
            insert_fight(fight[0], winner_id, loser_id, weight_class, title_fight, method, round_end, fight_time, outcome, url)

            # Fetch and store advanced stats for both fighters
            fight_url = fight[9]
            fightId = get_fight_id_from_url(fight_url)
            get_fight_stats(fight_url, fightId, winner_id, loser_id)
            

            # Update fighter records (wins, losses, draws)
            if outcome == "win":
                update_fighter_record(winner_id, "win")
                update_fighter_record(loser_id, "loss")
                update_elo(winner_id, loser_id, title_fight)  # Update Elo rating based on the outcome (only if win)
            elif outcome == "Draw":
                update_fighter_record(winner_id, "draw")
                update_fighter_record(loser_id, "draw")

        except Exception as e:
            print(f"Error processing fight {fight[0]}: {e}")

In [174]:
def get_fight_links(event_url):
    event_url_event = requests.get(event_url)
    event_soup = BeautifulSoup(event_url_event.text, "html.parser")
    
    fight_links = []
    fight_table = event_soup.find("table", class_="b-fight-details__table")
    
    if fight_table:
        for row in fight_table.find_all("tr")[1:]:  # Skip header row
            link = row.find("a", class_="b-link_style_black")
            if link and link.get("href"):
                fight_links.append("https://www.ufc.com" + link.get("href"))
    
    return fight_links

In [175]:
def get_fight_id(fighter_name):
    # Query to get fighter_id by name (adjust based on your database schema)
    cursor.execute("SELECT fighter_id FROM fighters WHERE name = ?", (fighter_name,))
    
    # Fetch the result
    result = cursor.fetchone()

    # If the fighter exists in the database, return their fighter_id
    if result:
        return result[0]  # Assuming fighter_id is the first column in the result
    else:
        # If the fighter is not found, return None or handle it differently
        print(f"Fighter '{fighter_name}' not found in the database.")
        return None

In [176]:
# Scrape fight data
url = requests.get('http://www.ufcstats.com/statistics/events/completed?page=all')
soup = BeautifulSoup(url.text, 'html.parser')
event_links = [link["href"] for link in soup.select(".b-statistics__table-row a")]
event_links.reverse()

In [177]:
"""
df = pd.read_csv("ufcEvent.csv")  # Make sure your CSV file is in the correct path


# Step 3: Prepare the SQL insert query for events table
insert_query = 
INSERT INTO events (name, date)
VALUES (%s, %s)


# Step 4: Convert DataFrame rows into a list of tuples
# Assuming 'event_name' and 'event_date' columns in the CSV
data = [(row['name'], row['date']) for row in df.to_dict(orient='records')]

# Step 5: Insert data in batches for performance
cursor.executemany(insert_query, data)

# Commit the transaction
conn.commit()

# Print how many rows were inserted
print(f"{cursor.rowcount} records inserted into the events table.")"
""
"""

'\ndf = pd.read_csv("ufcEvent.csv")  # Make sure your CSV file is in the correct path\n\n\n# Step 3: Prepare the SQL insert query for events table\ninsert_query = \nINSERT INTO events (name, date)\nVALUES (%s, %s)\n\n\n# Step 4: Convert DataFrame rows into a list of tuples\n# Assuming \'event_name\' and \'event_date\' columns in the CSV\ndata = [(row[\'name\'], row[\'date\']) for row in df.to_dict(orient=\'records\')]\n\n# Step 5: Insert data in batches for performance\ncursor.executemany(insert_query, data)\n\n# Commit the transaction\nconn.commit()\n\n# Print how many rows were inserted\nprint(f"{cursor.rowcount} records inserted into the events table.")"\n""\n'

In [178]:
# Step 4: Function to get or insert fighter ID into the 'fighters' table
def get_or_insert_fighter(name, weightclass):
    # Check if the fighter already exists
    cursor.execute("SELECT fighter_id FROM fighters WHERE name = %s", (name,))
    result = cursor.fetchone()

    if result:
        # If the fighter exists, return their ID
        return result[0]
    else:
        # If the fighter doesn't exist, insert into the fighters table
        cursor.execute(
            "INSERT INTO fighters (name, weightclass) VALUES (%s, %s)", 
            (name, weightclass)
        )
        conn.commit()
        # Return the ID of the newly inserted fighter
        return cursor.lastrowid


In [179]:
"""
df = pd.read_csv("fight_data.csv")

insert_fight_query = """ """
INSERT INTO fights (event_id, fighter1_id, fighter2_id, weight_class, title_fight, method, rounds, time, outcome, url)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
""" """


for row in df.itertuples(index=False):
    try:
        event_id = row.event_id
        fighter1_name = row.winner
        fighter2_name = row.loser
        weight_class = row.weight_class
        title_fight = row.title_fight
        method = row.true_method
        rounds = row.round_end
        time = row.time
        outcome = row.outcome
        url = row.fight_url
        
        # Step 6: Get or insert fighter IDs for both fighter1 and fighter2
        fighter1_id = get_or_insert_fighter(fighter1_name, weight_class)
        fighter2_id = get_or_insert_fighter(fighter2_name, weight_class)

        
        # Step 7: Insert fight data into the 'fights' table
        fight_data = (event_id, fighter1_id, fighter2_id, weight_class, title_fight, method, rounds, time, outcome, url)
        cursor.execute(insert_fight_query, fight_data)
        #fightId = get_fight_id_from_url(url)
        #get_fight_stats(url, fightId, fighter1_id, fighter2_id)
            
        if outcome == "win":
            update_fighter_record(fighter1_id, "win")
            update_fighter_record(fighter2_id, "loss")
            update_elo(fighter1_id, fighter2_id, title_fight)  # Update Elo rating based on the outcome (only if win)
        elif outcome == "draw":
            update_fighter_record(fighter1_id, "draw")
            update_fighter_record(fighter2_id, "draw")
        
        # Commit after every insertion to avoid memory overload
        conn.commit()
        
        
    except Exception as e:
        print(f"Error processing fight: {e}")
        continue"
    ""
"""

'\ndf = pd.read_csv("fight_data.csv")\n\ninsert_fight_query = \nINSERT INTO fights (event_id, fighter1_id, fighter2_id, weight_class, title_fight, method, rounds, time, outcome, url)\nVALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)\n\n\n\nfor row in df.itertuples(index=False):\n    try:\n        event_id = row.event_id\n        fighter1_name = row.winner\n        fighter2_name = row.loser\n        weight_class = row.weight_class\n        title_fight = row.title_fight\n        method = row.true_method\n        rounds = row.round_end\n        time = row.time\n        outcome = row.outcome\n        url = row.fight_url\n        \n        # Step 6: Get or insert fighter IDs for both fighter1 and fighter2\n        fighter1_id = get_or_insert_fighter(fighter1_name, weight_class)\n        fighter2_id = get_or_insert_fighter(fighter2_name, weight_class)\n\n        \n        # Step 7: Insert fight data into the \'fights\' table\n        fight_data = (event_id, fighter1_id, fighter2_id, weight_cl

In [180]:
# Load fight URLs from CSV
df = pd.read_csv("fight_data.csv")

# Loop through each fight URL
for index, row in df.iterrows():
    try:
        fight_url = row['fight_url']
        fight_id = get_fight_id_from_url(fight_url)  # Get fight ID

        if fight_id is None:
            print(f"Skipping fight {fight_url}: No fight ID found.")
            continue

        # Get or insert fighter IDs
        fighter1_id = get_or_insert_fighter(row['winner'], row['weight_class'])
        fighter2_id = get_or_insert_fighter(row['loser'], row['weight_class'])

        # Call get_fight_stats (which already inserts into the DB)
        get_fight_stats(fight_url, fight_id, fighter1_id, fighter2_id)
        conn.commit()

        print(f"Processed fight {fight_url} (Fight ID: {fight_id})")

    except Exception as e:
        print(f"Error processing fight {fight_url}: {e}")


Fighter Stats for Fighter 1: {'kd': '0', 'sig_str': '4', 'sig_str_pct': '100%', 'total_str': '11', 'td': '1', 'td_pct': '50%', 'sub_att': '0', 'rev': '0', 'ctrl': '--', 'head': '3', 'body': '0', 'leg': '1', 'distance': '0', 'clinch': '1', 'ground': '3'}
Fighter Stats for Fighter 2: {'kd': '0', 'sig_str': '1', 'sig_str_pct': '50%', 'total_str': '2', 'td': '0', 'td_pct': '---', 'sub_att': '0', 'rev': '0', 'ctrl': '--', 'head': '0', 'body': '1', 'leg': '0', 'distance': '0', 'clinch': '1', 'ground': '0'}
{'kd': '0', 'sig_str': '4', 'sig_str_pct': '100%', 'total_str': '11', 'td': '1', 'td_pct': '50%', 'sub_att': '0', 'rev': '0', 'ctrl': '--', 'head': '3', 'body': '0', 'leg': '1', 'distance': '0', 'clinch': '1', 'ground': '3'}
Sanitizing value: 0
Sanitizing value: 4
Sanitizing value: 100%
Sanitizing value: 11
Sanitizing value: 1
Sanitizing value: 50%
Sanitizing value: 0
Sanitizing value: 0
Sanitizing value: --
Sanitizing value: 3
Sanitizing value: 0
Sanitizing value: 1
Sanitizing value: 0
Sa

KeyboardInterrupt: 

In [ ]:
#can skip
"""
for event_url in event_links:
    event_url_event = requests.get(event_url)
    event_soup = BeautifulSoup(event_url_event.text, "html.parser")
    
    # Get event details
    event_name = event_soup.select_one(".b-content__title").text.strip()
    event_date_raw = event_soup.select_one(".b-list__box-list-item:first-child").text.strip()
    event_date_clean = re.search(r"([A-Za-z]+ \d{1,2}, \d{4})", event_date_raw)
    event_date = datetime.strptime(event_date_clean.group(1), "%B %d, %Y").strftime("%Y-%m-%d") if event_date_clean else None
    
    # Insert event
    event_id = insert_event(event_name, event_date)

    
    fight_table = event_soup.find("table", class_="b-fight-details__table")
    
    if fight_table:
        for row in fight_table.find_all("tr")[1:]:  # Skip header row
            cols = [col.text.strip() for col in row.find_all("td")]
            if len(cols) > 6:
                fight_link = row.get('data-link')

                print(f"Fight URL: {fight_link}")  # Debugging print

                fighters = [fighter.strip() for fighter in cols[1].split("\n") if fighter.strip()]
                winner = fighters[0] if fighters else "Unknown"
                loser = fighters[-1] if len(fighters) > 1 else "Unknown"
                weight_class = cols[6]
                method = [m.strip() for m in cols[7].split("\n") if m.strip()]
                true_method = method[0] if method else "Unknown"
                round_end = cols[8] if len(cols) > 8 else "Unknown"
                rounds = int(round_end.split(" ")[0])
                time = cols[9] if len(cols) > 9 else "Unknown"
                title_fight = "Yes" if row.find("img", src=lambda x: x and "belt.png" in x) else "No"
                
                # Determine the outcome
                outcome = "win"
                if "nc|" in cols[0]:
                    outcome = "nc"
                elif "draw" in cols[0]:
                    outcome = "draw"
                
                #print(f"Outcome: {outcome}")  # This will print the outcome for each fight
                print(f"Fight data: {event_id}, {winner}, {loser}, {weight_class}, {title_fight}, {true_method}, {round_end}, {time}, {outcome}, {fight_link}")  # Debugging print
                
                # Insert the fight data into the database
                fight_data.append([event_id, winner, loser, weight_class, title_fight, true_method, round_end, time, outcome, fight_link])"
                
columns = ["event_id", "winner", "loser", "weight_class", "title_fight", "true_method", "round_end", "time", "outcome", "fight_url"]

# Convert fight_data to a pandas DataFrame
df = pd.DataFrame(fight_data, columns=columns)

# Save DataFrame as a CSV file
df.to_csv("fight_data.csv", index=False)

print("Fight data saved to fight_data.csv")"
"""

'\nfor event_url in event_links:\n    event_url_event = requests.get(event_url)\n    event_soup = BeautifulSoup(event_url_event.text, "html.parser")\n    \n    # Get event details\n    event_name = event_soup.select_one(".b-content__title").text.strip()\n    event_date_raw = event_soup.select_one(".b-list__box-list-item:first-child").text.strip()\n    event_date_clean = re.search(r"([A-Za-z]+ \\d{1,2}, \\d{4})", event_date_raw)\n    event_date = datetime.strptime(event_date_clean.group(1), "%B %d, %Y").strftime("%Y-%m-%d") if event_date_clean else None\n    \n    # Insert event\n    event_id = insert_event(event_name, event_date)\n\n    \n    fight_table = event_soup.find("table", class_="b-fight-details__table")\n    \n    if fight_table:\n        for row in fight_table.find_all("tr")[1:]:  # Skip header row\n            cols = [col.text.strip() for col in row.find_all("td")]\n            if len(cols) > 6:\n                fight_link = row.get(\'data-link\')\n\n                print(f

In [ ]:
"""
# Load CSV into DataFrame
df = pd.read_csv("fight_data.csv")

# Ensure the first column is an integer (assuming the first column is 'id')
df.iloc[:, 0] = df.iloc[:, 0] - 795

# Save back to CSV
df.to_csv("modified_fight_data.csv", index=False)"
"""

'\n# Load CSV into DataFrame\ndf = pd.read_csv("fight_data.csv")\n\n# Ensure the first column is an integer (assuming the first column is \'id\')\ndf.iloc[:, 0] = df.iloc[:, 0] - 795\n\n# Save back to CSV\ndf.to_csv("modified_fight_data.csv", index=False)"\n'

In [ ]:
#fight_data = load_fight_data_from_csv('fight_data.csv')

In [ ]:
# Process fights and update Elo ratings
#process_fights(fight_data)

# Close connection
cursor.close()
conn.close()

InternalError: Unread result found